# Jupyter Lectures, First Year Project 2021

## Project 1 - Road collisions analysis, ITU Copenhagen

**Instructor: Michael Szell**

Course page: https://learnit.itu.dk/local/coursebase/view.php?ciid=590

This notebook contains all the code developed in the course lectures to wrangle and explore the data set from the project.

Contact: Michael Szell (misz@itu.dk)  
Created: 2021-01-29  
Last modified: 2021-02-16

<hr>

# Lecture 1: First data exploration

### Imports

In [ ]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import csv

import pandas as pd

%run -i ../scripts/functions.py

### Constants

Constants are written all caps: https://www.python.org/dev/peps/pep-0008/#constants

In [ ]:
PATH = {}
PATH["data_raw"] = "../data/raw/"
PATH["data_interim"] = "../data/interim/"
PATH["data_processed"] = "../data/processed/"
PATH["data_external"] = "../data/external/"

FILENAME = {}
FILENAME["accidents"] = "Road Safety Data - Accidents 2019.csv"
FILENAME["casualties"] = "Road Safety Data - Casualties 2019.csv"
FILENAME["vehicles"] = "Road Safety Data- Vehicles 2019.csv" # Note the inconsistent naming of the raw data files

TABLENAMES = ["accidents", "casualties", "vehicles"]

### Load raw data

The data were downloaded from here on Jan 4th: https://data.gov.uk/dataset/road-accidents-safety-data  
That page was updated afterwards (Jan 8th), so local and online data may be inconsistent.

We first explore one data table, the accidents.
We load the data set with `utf-8-sig` encoding: https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string

In [ ]:
dataraw = {}
dataraw["accidents"] = np.genfromtxt(PATH["data_raw"]+FILENAME["accidents"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')
dataraw["vehicles"] = np.genfromtxt(PATH["data_raw"]+FILENAME["vehicles"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')
dataraw["casualties"] = np.genfromtxt(PATH["data_raw"]+FILENAME["casualties"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')

In [ ]:
headerraw = {}
for tablename in TABLENAMES:
    headerraw[tablename] = list(dataraw[tablename].dtype.names)

It is always good to start with a "sneak preview":

In [ ]:
dataraw["accidents"][:5]

Reminder and documentation on structured arrays:  
https://numpy.org/devdocs/user/basics.rec.html

#### Insight: Mixed variable types

Accidents have mixed data types, including strins, floats, integers. Categorical variables are encoded as integers. The meaning of these categories can be looked up in: `../references/variable lookup.xls`

Number of records

In [ ]:
dataraw["accidents"].shape

Number of fields

In [ ]:
len(dataraw["accidents"].dtype)

In [ ]:
dataraw["accidents"].dtype

**"Data in the wild" puzzle: Why is the first field "\ufeffAccident_Index" and not "Accident_Index"?**

We load the data set with `utf-8-sig` encoding: https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string

Fields

In [ ]:
dataraw["accidents"].dtype.names

Homework: Explore the other two tables the same way.

https://en.wikipedia.org/wiki/Code_refactoring

<hr>

# Lecture 2: Command line wrangling and dealing with missing data

A faster way of getting basic insights into a new data set than by using numpy is by using command line tools.

Let's get a first overview using `head`. There are 3 data tables: Accidents, Casualties, and Vehicles.

In [ ]:
!head -n 6 "../data/raw/Road Safety Data - Accidents 2019.csv"
!head -n 6 "../data/raw/Road Safety Data - Casualties 2019.csv"
!head -n 6 "../data/raw/Road Safety Data- Vehicles 2019.csv"

### General insights

#### Link between data tables

Records between data tables are linked through their `Accident_Index`.

Looking at the first Accident_Index 2019010128300, we can see there seems to be a one-to-many relation between accident->casualty and accident->vehicle, meaning there can be multiple casualties and vehicles involved in one accident (makes sense).

https://en.wikipedia.org/wiki/One-to-many_(data_model)

#### Dimensions

Number of records

https://en.wikipedia.org/wiki/Wc_(Unix)

In [ ]:
!wc -l "../data/raw/Road Safety Data - Accidents 2019.csv"
!wc -l "../data/raw/Road Safety Data - Casualties 2019.csv"
!wc -l "../data/raw/Road Safety Data- Vehicles 2019.csv"

Number of fields (in first line)

https://www.geeksforgeeks.org/awk-command-unixlinux-examples/

In [ ]:
!head -1 "../data/raw/Road Safety Data - Accidents 2019.csv" | awk -F ',' '{print NF}'
!head -1 "../data/raw/Road Safety Data - Casualties 2019.csv" | awk -F ',' '{print NF}'
!head -1 "../data/raw/Road Safety Data- Vehicles 2019.csv" | awk -F ',' '{print NF}'

See and count all fields

https://en.wikipedia.org/wiki/Tr_(Unix)
https://en.wikipedia.org/wiki/Nl_(Unix)

In [ ]:
!head -1 "../data/raw/Road Safety Data - Accidents 2019.csv" | tr ',' '\n' | nl
!head -1 "../data/raw/Road Safety Data - Casualties 2019.csv" | tr ',' '\n' | nl
!head -1 "../data/raw/Road Safety Data- Vehicles 2019.csv" | tr ',' '\n' | nl

### Sanity checks

Has each record the same number of fields?

https://shapeshed.com/unix-uniq/  
https://www.putorius.net/uniq-command-linux.html

In [ ]:
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data - Accidents 2019.csv" | sort | uniq -d
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data - Casualties 2019.csv" | sort | uniq -d
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data- Vehicles 2019.csv" | sort | uniq -d

How many duplicate lines are there? (If more than 0, there could be a problem)

In [ ]:
!sort "../data/raw/Road Safety Data - Accidents 2019.csv" | uniq -d | wc -l
!sort "../data/raw/Road Safety Data - Casualties 2019.csv" | uniq -d | wc -l
!sort "../data/raw/Road Safety Data- Vehicles 2019.csv" | uniq -d | wc -l

More advanced stuff with `awk`: https://datafix.com.au/BASHing/2020-05-20.html

## Dealing with missing data

Using a masked array:  
https://numpy.org/devdocs/reference/maskedarray.baseclass.html#numpy.ma.MaskedArray

In [ ]:
dataraw_masked = {}
for tablename in TABLENAMES:
    dataraw_masked[tablename] = np.genfromtxt(PATH["data_raw"] + FILENAME[tablename], delimiter = ',', dtype = None, names = True, encoding='utf-8-sig', usemask = True)

In [ ]:
dataraw_masked["accidents"][:5]

In [ ]:
dataraw_masked["accidents"].mask[:5]

The first 5 rows seem complete. What about the rest?

In [ ]:
np.count_nonzero(dataraw_masked["accidents"].mask)

Oh oh, around 5% of rows have missing values. Which rows?

In [ ]:
row_incomplete = np.where(dataraw_masked["accidents"].mask)[0]
print(row_incomplete)

How many values in total?  
Which fields are missing?

In [ ]:
missingpositions = {}
missingvalues = 0
missingconfigurations = set()
for rowpos in row_incomplete:
    missingpositions_thisrow = list(np.where(list(dataraw_masked["accidents"].mask[rowpos]))[0])
    missingpositions[rowpos] = missingpositions_thisrow
    missingvalues += len(missingpositions_thisrow)
    missingconfigurations.add(tuple(missingpositions_thisrow))
    
missingfieldnames = [np.array(headerraw["accidents"])[c] for c in [list(b) for b in missingconfigurations]]

Summary of missing values:

In [ ]:
print("Incomplete rows: " + str(np.count_nonzero(dataraw_masked["accidents"].mask)))
print("Missing values: " + str(missingvalues))

print("\nMissing field configurations: " + str(missingconfigurations))
for i in missingfieldnames:
    print(i)

<hr>

# Lecture 3: Visual data exploration, Connecting tables, Association test

## Visual exploratory data analysis ("Plot your data")

### Bar plots of categorical variables

In [ ]:
headerraw["accidents"]

In [ ]:
field_name = "Accident_Severity"
field_categories = {1: "Fatal", 2: "Serious", 3: "Slight"}

categories, counts = np.unique(dataraw["accidents"][field_name], return_counts=True)

fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
axes.bar(categories, counts, fc="gray") # fc is the face color

axes.set_xlabel("")
axes.set_ylabel('Count')
axes.set_title(field_name)

axes.set_xticks(list(field_categories.keys()))
axes.set_xticklabels(field_categories.values());

We were lucky because the categories 1,2,3 were "nice". But usually they aren't, so we need to explicitly map to integers:

In [ ]:
field_name = "Speed_limit"
field_categories = {20: "20 MPH", 30: "30 MPH", 40: "40 MPH", 50: "50 MPH", 60: "60 MPH", 70: "70 MPH", -1: "N/A"}

categories, counts = np.unique(dataraw["accidents"][field_name], return_counts=True)

fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
axes.bar(range(len(categories)), counts, fc="gray") # fc is the face color

axes.set_xlabel("")
axes.set_ylabel('Count')
axes.set_title(field_name)
fig.autofmt_xdate(rotation=45)

axes.set_xticks(range(len(categories)))
axes.set_xticklabels([field_categories[c] for c in categories]);

Instead of copy-pasting code, let's write a function.

In [ ]:
field_name = "Light_Conditions"
field_categories = {1: "Daylight", 4: "Darkness - lights lit", 5: "Darkness - lights unlit", 6: "Darkness - no lighting", 7: "Darkness - lighting unknown", -1: "Data missing or out of range"}

barplot(dataraw["accidents"], field_name, field_categories)

Typing the variable lookup manually is cumbersome. Can we read the excel directly?  
pandas can: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

In [ ]:
variable_lookup = pd.read_excel("../references/variable lookup.xls", sheet_name = None)

In [ ]:
print(variable_lookup.keys())
print("\n")
print(variable_lookup["Introduction"])

### Histograms of numerical variables

In [ ]:
headerraw["casualties"]

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])
axes.hist(dataraw["casualties"]["Age_of_Casualty"], bins = np.linspace(-1,121, 123));

axes.set_ylabel('Counts')
axes.set_title("Age_of_Casualty");

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])
axes.hist(dataraw["vehicles"]["Age_of_Driver"], bins = np.linspace(0,121, 122));

axes.set_ylabel('Counts')
axes.set_title("Age_of_Driver");

### Categorical scatterplots

Scatterplots are good for relating two numerical variables. If we have one numerical versus one categorical variable, we can do a box plot. But could we also visualize all data points? Yes: https://seaborn.pydata.org/tutorial/categorical.html

In [ ]:
import seaborn as sns

In [ ]:
# Filter
mask = (dataraw["casualties"]["Casualty_Type"] == 0) | (dataraw["casualties"]["Casualty_Type"] == 1) | (dataraw["casualties"]["Casualty_Type"] == 9) &    (dataraw["casualties"]["Age_of_Casualty"] > -1)
mask[600:] = False # Restrict to at most 600 points

data_toplot = np.array([dataraw["casualties"]["Casualty_Type"][mask], dataraw["casualties"]["Age_of_Casualty"][mask]]).T

# Plot
fig = sns.catplot(x = 'Casualty Type', y = 'Age of Casualty', data = pd.DataFrame(data_toplot, columns=['Casualty Type', 'Age of Casualty']), kind = "swarm") # also show: violin
fig.set_xticklabels(["Pedestrian", "Cyclist", "Car occupant"]);

In [ ]:
mask = (dataraw["casualties"]["Casualty_Type"] == 0) | (dataraw["casualties"]["Casualty_Type"] == 1) | (dataraw["casualties"]["Casualty_Type"] == 9) & (dataraw["casualties"]["Age_of_Casualty"] > -1) & (dataraw["casualties"]["Sex_of_Casualty"] > -1)
mask[600:] = False

data_toplot = np.array([dataraw["casualties"]["Casualty_Type"][mask], dataraw["casualties"]["Age_of_Casualty"][mask], dataraw["casualties"]["Sex_of_Casualty"][mask]]).T

fig = sns.catplot(x='Casualty Type', y='Age of Casualty', hue='Sex of Casualty', data=pd.DataFrame(data_toplot, columns=['Casualty Type', 'Age of Casualty', 'Sex of Casualty']), kind="swarm") # also show: violin
fig.set_xticklabels(["Pedestrian", "Cyclist", "Car occupant"]);

## Connecting tables with np.isin()

**Question: How many babies and toddlers died on UK roads in June 2019?**

1. Get all the accident indices from the accidents table for June
2. Filter al casualties for those indices
3. Filter those casualties for ages 0-4

Numpy has the fucntion isin() to select for a list of indices: https://numpy.org/doc/stable/reference/generated/numpy.isin.html

In [ ]:
# 1
ind = [x["Accident_Index"] for x in dataraw["accidents"] if "/06/" in x["Date"]]
# 2
datafiltered = dataraw["casualties"][np.isin(dataraw["casualties"]["Accident_Index"], ind)]
# 3
datafiltered = datafiltered[(datafiltered["Age_of_Casualty"] >= 0) & (datafiltered["Age_of_Casualty"] <= 4)]

len(datafiltered)

**Question: Who killed them?**

Homework

## Association test between two categorical variables 
**(Pearson $\chi^2$ test of independence)**

Inspired by:  
https://peterstatistics.com/CrashCourse/3-TwoVarUnpair/NomNom/NomNom-2a-Test.html  
https://bit.ly/3kbwKEL

**Let us ask: Is there a statistically significant association between accident severity and speed limit?**  
We ask because speed limit is something that the city government can regulate.

### Hypothesis testing

We are now in the realm of [Statistical hypothesis testing](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing). In general, we must first state and compare two hypotheses:

- $H_0$ (null hypothesis): There is no statistically significant relationship between accident severity and speed limit.
- $H_\alpha$ (alternative hypothesis): There is a statistically significant relationship between accident severity and speed limit.

We must then 1) state+check statistical assumptions, 2) choose an appropriate test and test statistic $T$, 3) derive the distribution for the test statistic, 4) select a significance level $\alpha$, usually 0.01 or 0.05, 5) calculate the observed test statistic $t_{\mathrm obs}$, 6) calculate the [p-value](https://en.wikipedia.org/wiki/P-value). 

If the p-value $< \alpha$, then the null hypothesis will be rejected.

### Pearson $\chi^2$ test of independence

To test association between two categorical variables, one uses the [Pearson chi-square test of independence](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test). If the significance of this test (p-value) is below a significance level (typically 0.05), the two variables have a significant association.

The Pearson chi-square test should only be used if most cells have an expected count above 5, and the minimum expected count is at least 1.

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
mask = (dataraw["accidents"]["Speed_limit"] != -1)
severityspeed = np.array([dataraw["accidents"]["Speed_limit"][mask], dataraw["accidents"]["Accident_Severity"][mask]]).T
print(severityspeed.shape)
print(severityspeed)

We crosstabulate using pandas:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html

The cross tabulation is also known as contingency table.

In [ ]:
observed_pd = pd.crosstab(severityspeed[:, 0], severityspeed[:, 1], rownames = ["Speed limit"], colnames = ["Accident Severity"]) 
observed = observed_pd.to_numpy()
observed_pd

The idea is now to compare these observed values with expected values.  
The expected values can be calculated using:
\begin{equation*}
E_{i,j} = \frac{R_i \times C_j}{N}
\end{equation*}
The $E_{i,j}$ indicates the expected count in row i, column j. The $R_i$ is the row total of row i, and $C_j$ the column total of column j. The $N$ is the grand total.

In [ ]:
expected = np.zeros(observed.shape, dtype=int) # E, int for being more human-readable
colTotals = observed.sum(axis = 0) # C
rowTotals = observed.sum(axis = 1) # R
N = rowTotals.sum()

for i in range(observed.shape[0]):
    for j in range(observed.shape[1]):
        expected[i,j] = (rowTotals[i] * colTotals[j]) / N
        
expected

That was the manual way of doing it. `chi2_contingency()` can do it for us automatically:

In [ ]:
chiVal, pVal, df, expected = chi2_contingency(observed)
chiVal, pVal, df, expected.astype(int)

We now know that the association is significant, but how strong is it?  
Cramer's V, for example, can give an answer: https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V

The formula is:
\begin{equation*}
V=\sqrt{\frac{\chi^{2} / N}{\min (c-1, r-1)}}
\end{equation*}
where $c$ is the number of columns, $r$ is the number of rows.

In [ ]:
V = np.sqrt( (chiVal/N) / (min(observed.shape)-1) )
V

A V close to 0 implies a weak effect.

Let us visualize this and make a human-readable plot and report below.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))

severity_labels = ["Fatal", "Serious", "Slight"]
speed_categories = {20: "20 MPH", 30: "30 MPH", 40: "40 MPH", 50: "50 MPH", 60: "60 MPH", 70: "70 MPH"}
x = np.array(list(speed_categories.keys()))

for i, ax in enumerate(axes[0]):
    ax.plot(x, observed[:,i], 'ro-', label='Observed')
    ax.plot(x, expected[:,i], 'bo-', label='Expected')
    if i==0: 
        ax.set_ylabel('Casualties')
        ax.legend(loc='best');
    ax.set_title(severity_labels[i])
    ax.set_xticks(x)
    ax.set_xticklabels(list(speed_categories.values()))
    fig.autofmt_xdate(rotation=45)

for i, ax in enumerate(axes[1]):
    ax.plot(x, observed[:,i]/expected[:,i], 'go-')
    ax.plot(x, np.ones(x.shape), 'k:')
    
    if i==0: 
        ax.set_ylabel('Observed/Expected')
    ax.set_xticks(x)
    ax.set_xticklabels(list(speed_categories.values()))
    fig.autofmt_xdate(rotation=45)

**Conclusion**

An association only tells us about correlation, not causation. However, in this case, there is good reason to say that the speed of cars are a cause of the accident severity. Although Cramer's V is low, there is a very clear effect, especially for fatal collisions, $\chi^2$(10, N = 117456) = 1951.03, p < .001, V = 0.091.

The conclusion is therefore that **"Speed kills"**:  
Fatal collisions are over 2 times more likely than expected for high speeds (>60 mph), and are 2-3 times less likely than expected for low speed limits (<30 mph). An urban planning policy recommendation would therefore be: To reduce fatalities, reduce speed limits.

What about statistical tests for different combinations of numerical/categorical variables?
<img src="../references/flowchart-for-choosing-a-statistical-test.png" width="600px"/>

<hr>

# Lecture 4: Spatial filtering

## Filtering with external table

Let's select all rows for the city name:

We want to use this list of LSOA11 codes to restrict our accident data set.

Filter

Export

## Spatial filtering with shapely

Jupyter visualizes shapely objects!

Let's get all accident coordinates (from the whole UK)

`contains()` and `within()` check for point inclusion:

Limit vehicles and casualties to these AccidentIndices

What does it mean?  
See: https://www.computerhope.com/unix/udiff.htm

<hr>

# Lecture 5: Visualizing spatial data

Inspired by: https://alysivji.github.io/getting-started-with-folium.html

The heatmap is built with KDE:  
https://en.wikipedia.org/wiki/Kernel_density_estimation

We can also add automatic clusters: